In [1]:
import pandas as pd
import re

In [18]:
desc18 = pd.read_csv('ams_2018_202001290000_ams__cargodesc_2018__202001290000.csv',low_memory=False)
hs18 = pd.read_csv('ams_2018_202001290000_ams__tariff_2018__202001290000.csv',low_memory=False)

#inner join
df = desc18.merge(hs18, how='inner', left_on=["description_sequence_number","identifier", "container_number"], right_on=["description_sequence_number","identifier","container_number"])

#drop unnecessary columns
df = df.drop('identifier', axis=1)
df = df.drop('container_number', axis=1)
df = df.drop('description_sequence_number', axis=1)
df = df.drop('piece_count', axis=1)
df = df.drop('harmonized_value', axis=1)
df = df.drop('harmonized_weight', axis=1)
df = df.drop('harmonized_weight_unit', axis=1)

#drop the duplicates
df = df.drop_duplicates()

#drop missing values
df = df.dropna()

df['harmonized_number'] = df['harmonized_number'].apply(lambda x: x.replace(' ',''))

#remove corrupted HS codes
df['harmonized_number'].apply(lambda x: re.sub('[^0-9]','',x))

#drop HS-2 and HS-4
mask = (df['harmonized_number'].str.len() >= 6)
df = df.loc[mask]

#convert HTS to HS-6
df['harmonized_number'] = df['harmonized_number'].apply(lambda x: x[:6])

#drop first entries
df.drop_duplicates(subset ="description_text", 
                     keep = 'first', inplace = True)

df = df.reset_index()
df = df.drop(['index'],axis=1)

df.info()
df.to_csv('2018.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491405 entries, 0 to 491404
Data columns (total 2 columns):
description_text     491405 non-null object
harmonized_number    491405 non-null object
dtypes: object(2)
memory usage: 7.5+ MB


In [8]:
#merge tables for different years
df1 = pd.read_csv('2018.csv',low_memory=False)
df2 = pd.read_csv('2019.csv',low_memory=False)
df3 = pd.read_csv('2020.csv',low_memory=False)

df = [df1,df2,df3]
df = pd.concat(df)

df = df.reset_index()
df = df.drop(['index'],axis=1)

df.to_csv('2018-2020.csv', index=False)

df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1293762 entries, 0 to 1293761
Data columns (total 2 columns):
description_text     1293762 non-null object
harmonized_number    1293762 non-null object
dtypes: object(2)
memory usage: 19.7+ MB


,description_text,harmonized_number
0,WOODWORKING MACHINE AND SPARE PARTS H.S.CODE 8...,846591
1,WOODWORKING MACHINE AND SPARE PARTS PO NO. 751...,846591
2,"STAND, ZERO CLEARANCE THROAT PLATE, GLIDE PAD)...",846591
3,. . . . . . . .,846591
4,. . . . . .,846591


In [12]:
df = df.reset_index()
df = df.drop(['index'],axis=1)

df.to_csv('2018-2020.csv', index=False)